In [ ]:
# 1️⃣ Ensure dependencies are fresh
!pip install --upgrade prophet cmdstanpy

# 2️⃣ Force rebuild of the CmdStan backend
import cmdstanpy
cmdstanpy.install_cmdstan()

# 3️⃣ Verify Prophet works
from prophet import Prophet
import pandas as pd

# Dummy data test
df = pd.DataFrame({
    'ds': pd.date_range('2024-01-01', periods=30),
    'y': range(30)
})
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=5)
forecast = m.predict(future)

forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: prophet
    Found existing installation: prophet 1.1.7
    Uninstalling prophet-1.1.7:
      Successfully uninstalled prophet-1.1.7
CmdStan install directory: /root/.cmdstan
Installing CmdStan version: 2.37.0
Download successful, file: /tmp/tmp7vtptslw
Extracting distribution
Unpacked download as cmdstan-2.37.0
Building version cmdstan-2.37.0, may take several minutes, depending on your system.
Installed cmdstan-2.37.0
Test model compilation


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.


,ds,yhat,yhat_lower,yhat_upper
30,2024-01-31,30.014075,30.010689,30.017560
31,2024-02-01,31.014075,31.003083,31.025469
32,2024-02-02,32.014075,31.992028,32.036184
33,2024-02-03,33.014075,32.979400,33.048509
34,2024-02-04,34.014076,33.965595,34.064093


In [ ]:
!pip install flask plotly scikit-learn firebase-admin ultralytics flask-cors pyngrok tensorflow -q

# ============================================================
# SECTION 1: ALL IMPORTS
# ============================================================
import os
import re
import pickle
import threading
import io
import base64
import logging
import warnings
import json
import datetime
import time
import tempfile
import subprocess
from collections import Counter, defaultdict

# CV & ML Imports
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from ultralytics import YOLO
from prophet import Prophet
import tensorflow as tf
from transformers import BertTokenizer, TFAutoModel
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sklearn Imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors

# ------------------------------------------------------------
# Environment setup
# ------------------------------------------------------------
os.environ["TOKENIZERS_PARALLELISM"] = "false"
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# Flask Imports
from flask import Flask, render_template_string, request, jsonify, send_from_directory
from flask_cors import CORS
from pyngrok import ngrok
import plotly.express as px
import plotly.graph_objects as go
import plotly

# Firebase Imports
import firebase_admin
from firebase_admin import credentials, firestore

# Suppress warnings
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").setLevel(logging.ERROR)
logging.getLogger("werkzeug").setLevel(logging.WARNING)
warnings.filterwarnings("ignore")

# ============================================================
# SECTION 2: FLASK APP & CONFIGURATION
# ============================================================

# --- Create ONE shared Flask app ---
app = Flask(__name__)
CORS(app)

# --- Config for Trends Dashboard ---
SERVICE_ACCOUNT_PATH = "/content/drive/MyDrive/ML_project_Extra/sentiment_firebase.json"
COLLECTION = "trends"
KEYWORDS = [
    "Artificial Intelligence", "ChatGPT", "Climate Change",
    "Elections", "Healthcare", "Startups", "Cybersecurity"
]
PORT = 5077 # Main port for the combined app

# --- Config for YOLO Image Dashboard ---
YOLO_MODEL_PATH = "/content/drive/MyDrive/ML_project_Extra/image_model_save/best.pt"

# Emotion → Color map
color_map = {
    "Anger": (0, 0, 255),
    "Contempt": (128, 0, 128),
    "Disgust": (0, 255, 0),
    "Fear": (128, 0, 0),
    "Happy": (0, 255, 255),
    "Neutral": (192, 192, 192),
    "Sad": (255, 0, 0),
    "Surprise": (255, 140, 0),
}

# Sentiment mapping
sentiment_map = {
    "Anger": "Negative",
    "Contempt": "Negative",
    "Disgust": "Negative",
    "Fear": "Negative",
    "Happy": "Positive",
    "Neutral": "Neutral",
    "Sad": "Negative",
    "Surprise": "Positive",
}

emoji_map = {
    "Anger": "😡", "Contempt": "😒", "Disgust": "🤢",
    "Fear": "😱", "Happy": "😊", "Neutral": "😐",
    "Sad": "😢", "Surprise": "😮",
}

# --- Config for Text Emotion Dashboard ---
# Paths
# ------------------------------------------------------------
DATA_DIR =  "/content/drive/MyDrive/ML_project_Extra/model_save"
MODEL_PATH = os.path.join(DATA_DIR, "model_bert_v3", "bert_bilstm_hybrid_v3_ft.keras")
TOKENIZER_PATH = os.path.join(DATA_DIR, "model_bert_v3", "tokenizer.pkl")
LABEL_ENCODER_PATH = os.path.join(DATA_DIR, "model_bert_v3", "label_encoder.pkl")
BERT_TOKENIZER_DIR = os.path.join(DATA_DIR, "model_bert_v3")


IS_BERT_MODEL_READY = False
MAX_GLOVE_LEN = 150
BERT_MAX_LEN = 128

emotion_to_sentiment = {
    "sadness": "Negative 👎", "anger": "Negative 👎", "fear": "Negative 👎",
    "love": "Positive 👍", "joy": "Positive 👍", "surprise": "Neutral 😐"
}

# ============================================================
# SECTION 3A: YOLO MODEL INITIALIZATION
# ============================================================

# --- Load YOLO Model ---
try:
    # Use 'cpu' for a web server to avoid CUDA memory issues with multiple requests
    model = YOLO(YOLO_MODEL_PATH)
    model.to('cpu')
    print("✅ YOLO Model loaded successfully on CPU!")
except Exception as e:
    print(f"❌ Failed to load YOLO model: {e}")
    model = None

# ============================================================
# ============================================================
# SECTION 1: Custom BERT Layer (must match training)
# ============================================================
from tensorflow.keras.layers import Layer  # ✅ Add this line
from tensorflow.keras.models import load_model


@tf.keras.utils.register_keras_serializable()
class HF_Bert_Layer(Layer):
    def __init__(self, model_name="bert-base-uncased", trainable_layers=4, **kwargs):
        super().__init__(**kwargs)
        self.model_name = model_name
        self.trainable_layers = trainable_layers
        self.hf_model = TFAutoModel.from_pretrained(model_name, from_pt=True)
        self.hf_model.trainable = True

        try:
            # Freeze lower layers
            for layer in self.hf_model.bert.encoder.layer[:-trainable_layers]:
                layer.trainable = False
        except Exception:
            for layer in self.hf_model.encoder.layer[:-trainable_layers]:
                layer.trainable = False

    def call(self, inputs, training=False):
        input_ids, attention_mask = inputs
        outputs = self.hf_model(
            {
                "input_ids": tf.cast(input_ids, tf.int32),
                "attention_mask": tf.cast(attention_mask, tf.int32),
            },
            training=training
        )
        return outputs.last_hidden_state

    def get_config(self):
        config = super().get_config()
        config.update({
            "model_name": self.model_name,
            "trainable_layers": self.trainable_layers
        })
        return config


# ============================================================
# SECTION 2: Load Tokenizers, Encoders, etc.
# ============================================================
try:
    with open(TOKENIZER_PATH, "rb") as f:
        glove_tokenizer = pickle.load(f)
    with open(LABEL_ENCODER_PATH, "rb") as f:
        label_encoder = pickle.load(f)
    bert_tokenizer = BertTokenizer.from_pretrained(BERT_TOKENIZER_DIR)

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    print("✅ Tokenizers and encoders loaded successfully!")
except Exception as e:
    print(f"❌ Error loading preprocessing files: {e}")

# ============================================================
# SECTION 3: Load the Model Safely
# ============================================================
try:
    custom_objects = {"HF_Bert_Layer": HF_Bert_Layer}
    hybrid_model = load_model(MODEL_PATH, custom_objects=custom_objects, compile=False)
    print("✅ BERT Hybrid model loaded successfully!")
    IS_BERT_MODEL_READY = True
except Exception as e:
    print(f"❌ FATAL ERROR: Could not load BERT Hybrid model: {e}")
    hybrid_model = None

# ============================================================
# SECTION 4: Preprocessing and Prediction Functions
# ============================================================
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return " ".join(tokens)

def predict_emotion_sentiment(text):
    if not IS_BERT_MODEL_READY:
        return {"error": "Text analysis model is not ready."}

    # Preprocess text
    cleaned = clean_text(text)
    glove_seq = glove_tokenizer.texts_to_sequences([cleaned])
    glove_pad = pad_sequences(glove_seq, maxlen=MAX_GLOVE_LEN, padding='post', truncating='post')

    bert_enc = bert_tokenizer([text], padding='max_length', truncation=True,
                              max_length=BERT_MAX_LEN, return_tensors='np')
    input_ids = bert_enc["input_ids"]
    attention_mask = bert_enc["attention_mask"]

    # Predict
    preds = hybrid_model.predict([glove_pad, input_ids, attention_mask], verbose=0)
    probs = preds[0]

    emotion = label_encoder.inverse_transform([np.argmax(probs)])[0]
    confidence = float(np.max(probs) * 100)

    all_probs = {cls: float(p * 100) for cls, p in zip(label_encoder.classes_, probs)}

    # Optional: sentiment mapping (you can adjust this)
    emotion_to_sentiment = {
        "joy": "positive",
        "love": "positive",
        "anger": "negative",
        "sadness": "negative",
        "fear": "negative",
        "surprise": "neutral",
        "neutral": "neutral"
    }
    sentiment = emotion_to_sentiment.get(emotion, "neutral")

    return {
        "emotion": emotion,
        "confidence": round(confidence, 2),
        "sentiment": sentiment,
        "probabilities": all_probs,
        "original_text": text
    }



# ============================================================
# SECTION 3C: FIRESTORE INITIALIZATION
# ============================================================

# --- Initialize Firestore ---
def init_firestore(service_account_path=SERVICE_ACCOUNT_PATH):
    """Initialize Firestore App (if not already)"""
    if not os.path.exists(service_account_path):
        print(f"⚠️ Warning: Service account not found at {service_account_path}")
        return False
    try:
        if not firebase_admin._apps: # Check if any apps are initialized
            cred = credentials.Certificate(service_account_path)
            firebase_admin.initialize_app(cred)
            print("✅ Firebase app newly initialized.")
        else:
            print("✅ Firebase app already initialized.")
        return True
    except Exception as e:
        print(f"❌ Firestore initialization failed: {e}")
        return False

IS_DB_READY = init_firestore()
if IS_DB_READY:
    print("✅ Firestore is ready.")
else:
    print("❌ Firestore initialization failed - live queries will not work.")

# ============================================================
# SECTION 4: HELPER FUNCTIONS (TRENDS DASHBOARD)
# ============================================================

# --- [FIX 1 ADDED HERE] ---
def to_datetime_safe(ts_input):
    """Safely converts input to a pandas datetime, handling errors."""
    if ts_input is None:
        return pd.NaT
    # 'coerce' turns invalid dates into NaT (Not a Time)
    dt = pd.to_datetime(ts_input, errors='coerce')

    # Ensure it's timezone-aware (UTC) if it's not NaT and has no timezone
    if not pd.isna(dt) and dt.tzinfo is None:
        try:
            # Try to localize to UTC
            dt = dt.tz_localize('utc')
        except Exception:
            # Fallback if localization fails (e.g., ambiguous time)
            pass
    return dt


def fetch_docs(keyword=None, limit=None):
    """Fetch documents from Firestore with proper error handling"""
    if not IS_DB_READY:
        print("DB not ready, skipping fetch.")
        return pd.DataFrame() # Don't even try if init failed

    try:
        db = firestore.client() # <--- FIX: Get a fresh client every time
        q = db.collection(COLLECTION)
        if keyword:
            q = q.where("keyword", "==", keyword)

        docs = q.stream()
    except Exception as e:
        print(f"Error querying Firestore: {e}")
        return pd.DataFrame()

    rows = []
    for i, d in enumerate(docs):
        if limit and i >= limit:
            break
        rec = d.to_dict() or {}
        rec["_id"] = d.id
        for ts_field in ("timestamp", "first_seen_date", "first_seen_dt", "first_seen"):
            if ts_field in rec:
                rec[f"{ts_field}_dt"] = to_datetime_safe(rec.get(ts_field))
        for listf in ("entities", "keywords"):
            if listf not in rec or rec.get(listf) is None:
                rec[listf] = []
        rec["sentiment_score"] = pd.to_numeric(rec.get("sentiment_score"), errors="coerce")
        rows.append(rec)
    if not rows:
        return pd.DataFrame()
    df = pd.DataFrame(rows)
    if "first_seen_date_dt" in df.columns:
        df["first_seen_date"] = df["first_seen_date_dt"].dt.date
    elif "first_seen_dt" in df.columns:
        df["first_seen_date"] = df["first_seen_dt"].dt.date
    elif "timestamp_dt" in df.columns:
        df["first_seen_date"] = df["timestamp_dt"].dt.date
    return df

def load_trends_df(keyword=None, limit=None):
    if keyword is None:
        frames = []
        for kw in KEYWORDS:
            df_kw = fetch_docs(keyword=kw, limit=limit)
            if not df_kw.empty:
                if "keyword" not in df_kw.columns:
                    df_kw["keyword"] = kw
                frames.append(df_kw)
        return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    else:
        df_kw = fetch_docs(keyword=keyword, limit=limit)
        if not df_kw.empty and "keyword" not in df_kw.columns:
            df_kw["keyword"] = keyword
        return df_kw

# --- (All other Phase helper functions: phase1_analysis, phase2_analysis, etc. go here) ---
# --- [OMITTED FOR BREVITY - YOUR FULL CODE HAS THEM] ---
# --- Phase 1: Single Keyword ---
def phase1_analysis(keyword):
    df = load_trends_df(keyword)
    charts = {}
    if df.empty:
        return "<p class='alert alert-warning'>No data found for this keyword.</p>", charts
    sent_counts = df["sentiment"].fillna("unknown").value_counts().reset_index()
    sent_counts.columns = ['sentiment', 'count']
    fig_sent = px.pie(sent_counts, values='count', names='sentiment',
                      title=f"Sentiment Distribution: {keyword}", hole=0.3)
    fig_sent.update_traces(textposition='inside', textinfo='percent+label')
    charts['sentiment'] = fig_sent
    if "emotion" in df.columns:
        emo_counts = df["emotion"].fillna("unknown").value_counts().reset_index()
        emo_counts.columns = ['emotion', 'count']
        fig_emo = px.bar(emo_counts, x='emotion', y='count',
                           title=f"Emotion Distribution: {keyword}",
                           color_discrete_sequence=['#FF6B6B'])
        charts['emotion'] = fig_emo
    all_entities = [e for sub in df.get("entities", []) for e in (sub or [])]
    if all_entities:
        ent_counts = Counter(all_entities).most_common(15)
        ents, vals = zip(*ent_counts)
        fig_ent = px.bar(x=list(vals), y=list(ents), orientation='h',
                         title="Top 15 Entities", color_discrete_sequence=['#4ECDC4'])
        fig_ent.update_layout(yaxis={'categoryorder':'total ascending'})
        charts['entities'] = fig_ent
    all_keywords = [k for sub in df.get("keywords", []) for k in (sub or [])]
    if all_keywords:
        kw_counts = Counter(all_keywords).most_common(15)
        kws, vals = zip(*kw_counts)
        fig_kw = px.bar(x=list(vals), y=list(kws), orientation='h',
                        title="Top 15 Keywords", color_discrete_sequence=['#95E1D3'])
        fig_kw.update_layout(yaxis={'categoryorder':'total ascending'})
        charts['keywords'] = fig_kw
    table_html = df.head(10).to_html(index=False, classes="table table-sm table-striped table-hover")
    return table_html, charts

# --- Phase 2: Comparison ---
def phase2_analysis(kw1, kw2, top_n=10):
    df1 = load_trends_df(kw1)
    df2 = load_trends_df(kw2)
    charts = {}
    overlap_html = ""
    if df1.empty and df2.empty:
        return ("<p class='alert alert-warning'>No data for both keywords.</p>",
                "<p class='alert alert-warning'>No data for both keywords.</p>",
                charts, overlap_html)
    if df1.empty:
        return ("<p class='alert alert-warning'>No data for first keyword.</p>",
                df2.head(10).to_html(index=False, classes='table table-striped'),
                charts, overlap_html)
    if df2.empty:
        return (df1.head(10).to_html(index=False, classes='table table-striped'),
                "<p class='alert alert-warning'>No data for second keyword.</p>",
                charts, overlap_html)

    sent1 = df1["sentiment"].fillna("unknown").value_counts().reset_index()
    sent1.columns = ['sentiment', 'count']
    sent2 = df2["sentiment"].fillna("unknown").value_counts().reset_index()
    sent2.columns = ['sentiment', 'count']
    charts['sentiment'] = {
        'kw1': px.pie(sent1, values='count', names='sentiment', title=f"Sentiment: {kw1}", hole=0.3),
        'kw2': px.pie(sent2, values='count', names='sentiment', title=f"Sentiment: {kw2}", hole=0.3)
    }
    c1 = df1["emotion"].fillna("unknown").value_counts().reset_index()
    c1.columns = ['emotion', 'count']
    c2 = df2["emotion"].fillna("unknown").value_counts().reset_index()
    c2.columns = ['emotion', 'count']
    charts['emotion'] = {
        'kw1': px.bar(c1, x='emotion', y='count', title=f"Emotion: {kw1}", color_discrete_sequence=['#FFD93D']),
        'kw2': px.bar(c2, x='emotion', y='count', title=f"Emotion: {kw2}", color_discrete_sequence=['#6BCB77'])
    }
    all_entities1 = [e for sub in df1.get("entities", []) for e in (sub or [])]
    all_entities2 = [e for sub in df2.get("entities", []) for e in (sub or [])]
    ent_counts1 = Counter(all_entities1).most_common(top_n)
    ent_counts2 = Counter(all_entities2).most_common(top_n)
    charts['entities'] = {}
    if ent_counts1:
        ents1, vals1 = zip(*ent_counts1)
        charts['entities']['kw1'] = px.bar(x=list(vals1), y=list(ents1), orientation='h', title=f"Top Entities: {kw1}", color_discrete_sequence=['#4ECDC4'])
        charts['entities']['kw1'].update_layout(yaxis={'categoryorder':'total ascending'})
    if ent_counts2:
        ents2, vals2 = zip(*ent_counts2)
        charts['entities']['kw2'] = px.bar(x=list(vals2), y=list(ents2), orientation='h', title=f"Top Entities: {kw2}", color_discrete_sequence=['#FF6B6B'])
        charts['entities']['kw2'].update_layout(yaxis={'categoryorder':'total ascending'})

    all_keywords1 = [k for sub in df1.get("keywords", []) for k in (sub or [])]
    all_keywords2 = [k for sub in df2.get("keywords", []) for k in (sub or [])]
    set_kw1, set_kw2 = set(all_keywords1), set(all_keywords2)
    if set_kw1 or set_kw2:
        jaccard = 0.0
        if (set_kw1 | set_kw2):
            jaccard = len(set_kw1 & set_kw2) / len(set_kw1 | set_kw2)
        overlap_html += f"<div class='alert alert-info'><strong>🔗 Keyword Jaccard Similarity:</strong> {jaccard:.2%}</div>"
        common_sample = list(set_kw1 & set_kw2)[:10]
        if common_sample:
            overlap_html += f"<div class='alert alert-success'><strong>✅ Common Keywords:</strong> {', '.join(common_sample)}</div>"

    table1 = df1.head(10).to_html(index=False, classes='table table-striped table-sm')
    table2 = df2.head(10).to_html(index=False, classes='table table-striped table-sm')
    return table1, table2, charts, overlap_html

# --- Phase 4: Trends ---
def fetch_historical(keyword, days=50):
    df = load_trends_df(keyword)
    if df is None or df.empty: return None
    if "timestamp_dt" in df.columns:
        df["timestamp_dt"] = pd.to_datetime(df["timestamp_dt"], utc=True, errors="coerce")
    else:
        df["timestamp_dt"] = pd.to_datetime(
            df.get("first_seen_dt") if "first_seen_dt" in df.columns else None,
            utc=True, errors="coerce"
        )
    df = df.dropna(subset=["timestamp_dt"])
    if df.empty: return None
    df["date"] = df["timestamp_dt"].dt.date
    cutoff = datetime.date.today() - datetime.timedelta(days=days)
    df = df[df["date"] >= cutoff]
    return df.sort_values("timestamp_dt") if not df.empty else None

def compute_trends(keyword, days=50):
    df = fetch_historical(keyword, days)
    if df is None or df.empty: return None
    trend_df = df.groupby("date").agg(
        avg_sentiment=("sentiment_score", "mean"),
        article_count=("title", "count")
    ).reset_index()
    trend_df["sentiment_change"] = trend_df["avg_sentiment"].diff().fillna(0)
    return trend_df

def sentiment_alerts(trend_df, threshold_drop=0.05, threshold_rise=0.05):
    alerts = []
    for i, row in trend_df.iterrows():
        date = row["date"]
        sentiment_change = row.get("sentiment_change", 0)
        if sentiment_change <= -threshold_drop:
            alerts.append(f"⚠️ Sentiment dropped {sentiment_change:.2f} on {date}")
        elif sentiment_change >= threshold_rise:
            alerts.append(f"📈 Sentiment rose {sentiment_change:.2f} on {date}")
    return alerts

def run_phase4(keywords, days=50):
    results = {}
    for kw in keywords:
        trend_df = compute_trends(kw, days)
        if trend_df is not None and not trend_df.empty:
            alerts = sentiment_alerts(trend_df)
            trend_df_display = trend_df.tail(7)
            trend_html = trend_df_display.to_html(index=False, classes='table table-sm table-bordered')
            results[kw] = {"trend_html": trend_html, "alerts": alerts}
    return results

# --- Phase 5: Storyline ---
def fetch_last_5_days(keyword):
    df = load_trends_df(keyword)
    if df.empty: return pd.DataFrame()
    if "first_seen_dt" not in df.columns or df["first_seen_dt"].isna().all():
        if "timestamp_dt" in df.columns:
            df["first_seen_dt"] = pd.to_datetime(df["timestamp_dt"], utc=True, errors="coerce")
        else:
            df["first_seen_dt"] = pd.NaT
    df = df.dropna(subset=["first_seen_dt"])
    if df.empty: return pd.DataFrame()
    df = df.sort_values("first_seen_dt", ascending=False)
    last_5_dates = df["first_seen_dt"].dt.date.drop_duplicates().head(5)
    df = df[df["first_seen_dt"].dt.date.isin(last_5_dates)]
    return df

def textrank_summary(texts, top_n=3):
    if not texts: return []
    vectorizer = TfidfVectorizer(stop_words="english", max_features=200)
    try:
        tfidf_matrix = vectorizer.fit_transform(texts)
        similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
        scores = similarity_matrix.sum(axis=1)
        top_indices = scores.argsort()[-top_n:][::-1]
        return [texts[i] for i in top_indices]
    except Exception:
        return texts[:top_n]

def create_combined_story(df, keyword):
    if df.empty:
        return f"<div class='alert alert-info'>ℹ️ No recent articles found for '{keyword}' in the last 5 days.</div>"
    story_html = []
    df = df.sort_values("first_seen_dt")
    sentiment_counts = Counter(df["sentiment"].fillna("neutral"))
    emotion_counts = Counter(df["emotion"].fillna("unknown"))
    dominant_sentiment = sentiment_counts.most_common(1)[0][0] if sentiment_counts else "neutral"
    dominant_emotion = emotion_counts.most_common(1)[0][0] if emotion_counts else "unknown"
    total_articles = len(df)
    num_days = len(df['first_seen_dt'].dt.date.unique())
    story_html.append(f"<h3 class='text-primary mt-4'>{keyword} - Timeline & Themes</h3>")
    story_html.append(
        f"<p class='lead'>Over the last <strong>{num_days} days</strong>, we tracked <strong>{total_articles} articles</strong> "
        f"about <strong>{keyword}</strong>. Overall sentiment: "
        f"<span class='badge bg-info'>{dominant_sentiment}</span>, "
        f"dominant emotion: <span class='badge bg-warning'>{dominant_emotion}</span>.</p>"
    )
    story_html.append("<h4 class='mt-4'>📅 Day-by-Day Timeline</h4>")
    for date, group in df.groupby(df["first_seen_dt"].dt.date):
        day_str = pd.to_datetime(date).strftime("%A, %B %d")
        count = len(group)
        sentiments = group["sentiment"].fillna("neutral").value_counts().to_dict()
        top_headlines = group["title"].dropna().head(3).tolist()
        story_html.append(f"<div class='card mb-3'>")
        story_html.append(f"<div class='card-header bg-light'><strong>{day_str}</strong> ({count} articles)</div>")
        story_html.append("<div class='card-body'><ul>")
        story_html.append(f"<li><strong>Sentiment:</strong> {', '.join([f'{k}: {v}' for k, v in sentiments.items()])}</li>")
        if top_headlines:
            story_html.append("<li><strong>Key stories:</strong><ol>")
            for h in top_headlines:
                story_html.append(f"<li>{h}</li>")
            story_html.append("</ol></li>")
        story_html.append("</ul></div></div>")
    story_html.append("<h4 class='mt-4'>🎯 Major Themes</h4>")
    story_html.append("<ul class='list-group mb-3'>")
    df["theme"] = df["sentiment"].fillna("neutral") + "_" + df["emotion"].fillna("unknown")
    for theme, group in df.groupby("theme"):
        representative = group.iloc[0]
        title = representative.get("title", "") or ""
        sentiment, emotion = theme.split("_")
        story_html.append(
            f"<li class='list-group-item'><strong>{len(group)} article(s)</strong> with "
            f"<span class='badge bg-secondary'>{sentiment}</span> sentiment "
            f"(<span class='badge bg-info'>{emotion}</span> emotion): "
            f"\"{title[:120]}{'...' if len(title) > 120 else ''}\"</li>"
        )
    story_html.append("</ul>")
    story_html.append("<h4 class='mt-4'>⭐ Top Highlights</h4>")
    story_html.append("<ol class='list-group list-group-numbered mb-3'>")
    texts = [
        f"{row.get('title', '')} (Source: {row.get('source', 'unknown')}, "
        f"Sentiment: {row.get('sentiment', 'unknown')})"
        for _, row in df.iterrows()
    ]
    for art in textrank_summary(texts, top_n=min(5, len(texts))):
        story_html.append(f"<li class='list-group-item'>{art}</li>")
    story_html.append("</ol>")
    story_html.append("<h4 class='mt-4'>💡 Key Insights</h4>")
    story_html.append("<ul class='list-group mb-3'>")
    if sentiment_counts.get("positive", 0) > sentiment_counts.get("negative", 0) * 1.5:
        story_html.append("<li class='list-group-item list-group-item-success'>Coverage has been notably positive.</li>")
    elif sentiment_counts.get("negative", 0) > sentiment_counts.get("positive", 0) * 1.5:
        story_html.append("<li class='list-group-item list-group-item-danger'>Coverage has taken a negative turn.</li>")
    else:
        story_html.append("<li class='list-group-item list-group-item-warning'>Sentiment remains mixed.</li>")
    story_html.append(f"<li class='list-group-item'>Information from <strong>{df['source'].nunique()}</strong> unique sources.</li>")
    try:
        peak_day = df.groupby(df["first_seen_dt"].dt.date).size().idxmax()
        peak_count = df.groupby(df["first_seen_dt"].dt.date).size().max()
        story_html.append(
            f"<li class='list-group-item list-group-item-info'>Peak coverage: "
            f"<strong>{pd.to_datetime(peak_day).strftime('%B %d')}</strong> ({peak_count} articles)</li>"
        )
    except Exception: pass
    story_html.append("</ul>")
    return "\n".join(story_html)

# --- Phase 6: Clustering ---
def build_features_better(df):
    for col in ("content", "sentiment_score"):
        if col not in df.columns:
            df[col] = ""
    sent = pd.to_numeric(df.get("sentiment_score", 0), errors="coerce").fillna(0).to_numpy()
    text_len = df["content"].fillna("").str.len().to_numpy()
    cap_count = df["content"].fillna("").str.findall(r"\b[A-Z][a-zA-Z]+\b").str.len().fillna(0).to_numpy()
    X = np.c_[sent, text_len, cap_count]
    return df, pd.DataFrame(X, columns=["sentiment", "text_len", "cap_count"])

def choose_eps(X_scaled, min_samples=5, clip=(0.2, 2.0)):
    try:
        nbrs = NearestNeighbors(n_neighbors=min_samples).fit(X_scaled)
        dists, _ = nbrs.kneighbors(X_scaled)
        kdist = np.sort(dists[:, -1])
        n = len(kdist)
        if n < min_samples * 2: return 0.5, kdist
        x = np.arange(n)
        x0, y0 = 0, kdist[0]
        x1, y1 = n - 1, kdist[-1]
        num = np.abs((y1 - y0) * x - (x1 - x0) * kdist + x1 * y0 - y1 * x0)
        den = np.sqrt((y1 - y0) ** 2 + (x1 - x0) ** 2)
        knee_idx = int(np.argmax(num / den))
        eps = float(kdist[knee_idx])
        return float(np.clip(eps, *clip)), kdist
    except Exception:
        return 0.5, np.array([])

def run_dbscan_pipeline_base64(keywords, n_cols=3):
    img_base64_list = []
    if not keywords: return img_base64_list
    n_rows = (len(keywords) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 5 * n_rows))
    axes = np.array(axes).reshape(-1)
    for i, kw in enumerate(keywords):
        ax = axes[i]
        df_kw = load_trends_df(kw)
        if df_kw.empty:
            ax.set_title(f"{kw}\n(no data)"); ax.axis("off"); continue
        df_kw, X = build_features_better(df_kw)
        scaler = StandardScaler()
        try:
            X_scaled = scaler.fit_transform(X)
        except Exception:
            X_scaled = X.fillna(0).values
        eps, _ = choose_eps(X_scaled, min_samples=5, clip=(0.2, 2.0))
        dbscan = DBSCAN(eps=eps, min_samples=5)
        labels = dbscan.fit_predict(X_scaled)
        n_out = int(np.sum(labels == -1))
        n_pts = len(labels)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        perplexity = max(5, min(30, max(5, n_pts // 3)))
        try:
            X_tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity).fit_transform(X_scaled)
        except Exception:
            X_tsne = np.random.RandomState(42).randn(n_pts, 2)
        cluster_ids = sorted([c for c in set(labels) if c != -1])
        palette = plt.cm.tab10(np.linspace(0, 1, max(1, len(cluster_ids))))
        color_map_plot = {cid: palette[j] for j, cid in enumerate(cluster_ids)}
        point_colors = ["red" if l == -1 else color_map_plot.get(l, (0.3, 0.3, 0.3)) for l in labels]
        ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c=point_colors, s=45, edgecolors="k", linewidths=0.3, alpha=0.7)
        ax.set_title(f"{kw}\nClusters={n_clusters}, Outliers={n_out}/{n_pts}", fontsize=10)
        ax.set_xlabel("t-SNE 1"); ax.set_ylabel("t-SNE 2"); ax.grid(True, alpha=0.3)
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")
    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format="png", dpi=120, bbox_inches="tight")
    buf.seek(0)
    img_base64_list.append(base64.b64encode(buf.read()).decode("utf-8"))
    buf.close()
    plt.close(fig)
    return img_base64_list

# --- Phase 7: Forecasting ---
def generate_forecast(df, days=7):
    df2 = df.copy()
    if "timestamp_dt" not in df2.columns or df2["timestamp_dt"].isna().all():
        df2["timestamp_dt"] = pd.to_datetime(
            df2.get("first_seen_dt") if "first_seen_dt" in df2.columns else df2.get("timestamp"),
            utc=True, errors="coerce"
        )
    df2 = df2.dropna(subset=["timestamp_dt"])
    if df2.empty: return None, None, None
    df2["date"] = df2["timestamp_dt"].dt.date
    agg_dict = {"sentiment_score": "mean", "keyword": "count"}
    if "emotion" in df2.columns:
        agg_dict["emotion"] = lambda x: x.value_counts().index[0] if len(x) > 0 else "neutral"
    daily = df2.groupby("date").agg(agg_dict).reset_index()
    daily = daily.rename(columns={"date": "ds", "sentiment_score": "y", "keyword": "volume"})
    if "emotion" in daily.columns:
        daily = daily.rename(columns={"emotion": "dominant_emotion"})
    else:
        daily["dominant_emotion"] = "neutral"
    daily["ds"] = pd.to_datetime(daily["ds"])
    emotion_map_prophet = {e: i for i, e in enumerate(daily["dominant_emotion"].unique())}
    daily["emotion_code"] = daily["dominant_emotion"].map(emotion_map_prophet)
    if len(daily) < 5: return None, None, None
    m = Prophet(daily_seasonality=True, weekly_seasonality=True, interval_width=0.8)
    try:
        m.add_regressor("volume"); m.add_regressor("emotion_code")
    except Exception: pass
    m.fit(daily[["ds", "y", "volume", "emotion_code"]])
    future = m.make_future_dataframe(periods=days)
    future["volume"] = int(daily["volume"].iloc[-1]) if "volume" in daily.columns else 0
    future["emotion_code"] = int(daily["emotion_code"].iloc[-1]) if "emotion_code" in daily.columns else 0
    forecast = m.predict(future)
    forecast["yhat"] = forecast["yhat"].clip(-1, 1)
    forecast["yhat_lower"] = forecast["yhat_lower"].clip(-1, 1)
    forecast["yhat_upper"] = forecast["yhat_upper"].clip(-1, 1)
    return m, forecast, daily

def run_forecast_pipeline_base64(keywords, days=7, ncols=3):
    results = []
    for kw in keywords:
        df = load_trends_df(keyword=kw)
        if df.empty: continue
        m, forecast, daily = generate_forecast(df, days=days)
        if forecast is None: continue
        results.append((kw, daily, forecast))
    if not results: return []
    n = len(results)
    nrows = int(np.ceil(n / ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(6 * ncols, 4 * nrows), squeeze=False)
    axes = axes.flatten()
    for i, (kw, daily, forecast) in enumerate(results):
        ax = axes[i]
        ax.set_title(f"Sentiment Forecast: {kw}", fontsize=11, fontweight='bold')
        ax.plot(daily["ds"], daily["y"], "o-", color="blue", label="Actual", linewidth=2, markersize=4)
        ax.plot(forecast["ds"], forecast["yhat"], color="orange", label="Forecast", linewidth=2)
        ax.fill_between(
            forecast["ds"],
            forecast["yhat_lower"],
            forecast["yhat_upper"],
            color="orange", alpha=0.2, label="Confidence Interval"
        )
        merged = pd.merge(daily, forecast[["ds", "yhat_lower", "yhat_upper"]], on="ds", how="inner")
        anomalies = merged[(merged["y"] < merged["yhat_lower"]) | (merged["y"] > merged["yhat_upper"])]
        if not anomalies.empty:
            ax.scatter(anomalies["ds"], anomalies["y"], color="red", s=80, label="Anomalies", zorder=5, edgecolors='black')
        ax.axhline(0, color="black", linestyle="--", linewidth=0.8, alpha=0.6)
        ax.set_xlabel("Date", fontsize=9); ax.set_ylabel("Sentiment (-1 to 1)", fontsize=9)
        ax.tick_params(axis='x', rotation=45, labelsize=8)
        ax.grid(True, alpha=0.3); ax.legend(fontsize=7, loc='best')
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])
    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format="png", dpi=120, bbox_inches="tight")
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode("utf-8")
    buf.close()
    plt.close(fig)
    return [img_base64]


# ============================================================
# SECTION 5: PRE-GENERATION OF HEAVY DATA
# ============================================================
print("🔄 Pre-generating data for phases 5, 6, and 7...")
phase5_html = {}
for kw in KEYWORDS[:7]:
    df = fetch_last_5_days(kw)
    if df is not None and not df.empty:
        phase5_html[kw] = create_combined_story(df, kw)

phase6_images = run_dbscan_pipeline_base64(KEYWORDS[:7])
phase7_images = run_forecast_pipeline_base64(KEYWORDS[:7], days=7)
print("✅ Pre-generation complete!")


# ============================================================
# SECTION 6: HTML TEMPLATES
# ============================================================

# --- (MODIFIED) Template 1: Landing Page (Links to /text_analyzer) ---
LANDING_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Online News</title>
<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;700&display=swap" rel="stylesheet">
<style>
    :root {
        --primary-deep-green: #1a535c;
        --primary-teal: #2a9d8f;
        --accent-gold: #f7b731;
        --accent-gold-hover: #e6a82a;
        --dark-bg: #112022;
        --light-bg: #f4f8f7;
    }
    * { margin: 0; padding: 0; box-sizing: border-box; }
    body { font-family: 'Poppins', sans-serif; line-height: 1.6; scroll-behavior: smooth; }
    nav {
        position: fixed;
        width: 100%;
        background: rgba(26, 83, 92, 0.8);
        color: white;
        display: flex;
        justify-content: center;
        padding: 15px 0;
        z-index: 1000;
        backdrop-filter: blur(5px);
    }
    nav a {
        color: white;
        margin: 0 20px;
        text-decoration: none;
        font-weight: bold;
        transition: color 0.3s;
    }
    nav a:hover { color: var(--accent-gold); }
    .hero {
        height: 100vh;
        background: url('https://images.pexels.com/photos/6236039/pexels-photo-6236039.jpeg') no-repeat center center/cover;
        display: flex;
        flex-direction: column;
        justify-content: center;
        align-items: center;
        text-align: center;
        color: white;
        position: relative;
        overflow: hidden;
    }
    .hero::after {
        content: '';
        position: absolute;
        top: 0; left: 0;
        width: 100%; height: 100%;
        background: rgba(10, 40, 40, 0.7); /* Dark green overlay */
        z-index: 0;
    }
    .hero h1, .hero p, .hero button {
        position: relative;
        z-index: 1;
        opacity: 0;
        animation: fadeInUp 1.5s ease forwards;
    }
    .hero h1 { font-size: 3rem; margin-bottom: 20px; animation-delay: 0.3s; }
    .hero p { font-size: 1.2rem; margin-bottom: 30px; max-width: 600px; animation-delay: 0.6s; }
    .hero button {
        padding: 15px 40px;
        font-size: 20px;
        border: none;
        border-radius: 10px;
        background-color: var(--accent-gold);
        color: var(--dark-bg);
        font-weight: bold;
        cursor: pointer;
        transition: transform 0.3s, background-color 0.3s;
        animation-delay: 0.9s;
    }
    .hero button:hover {
        background-color: var(--accent-gold-hover);
        transform: scale(1.1);
    }
    @keyframes fadeInUp {
        from { transform: translateY(40px); opacity: 0; }
        to { transform: translateY(0); opacity: 1; }
    }
    section {
        padding: 80px 20px;
        text-align: center;
    }
    .features {
        background: var(--light-bg);
        display: flex;
        justify-content: center;
        flex-wrap: wrap;
    }
    .feature {
        background: #fff;
        margin: 20px;
        padding: 30px;
        border-radius: 15px;
        flex: 1 1 250px;
        max-width: 300px;
        box-shadow: 0 5px 15px rgba(0,0,0,0.1);
        opacity: 0;
        transform: translateY(40px);
        transition: all 0.6s ease;
    }
    .feature.visible {
        opacity: 1;
        transform: translateY(0);
    }
    .feature:hover {
        transform: translateY(-5px);
        box-shadow: 0 8px 20px rgba(0,0,0,0.2);
    }
    .feature h2 { color: var(--primary-deep-green); }
    footer {
        background: var(--primary-deep-green);
        color: white;
        padding: 40px 20px;
        text-align: center;
    }
    footer h3 { color: var(--accent-gold); }
    footer p {
        margin-top: 10px;
        font-size: 14px;
        color: #bbb;
    }
</style>
</head>
<body>
<nav>
    <a href="#hero">Home</a>
    <a href="#features">Features</a>
    <a href="#footer">Contact</a>
</nav>
<div class="hero" id="hero">
    <h1>Welcome to Our Website!</h1>
    <p>An AI-driven system that analyzes, tracks, and forecasts<br> sentiment trends from collected text data.</p>
    <button onclick="window.location.href='/dashboard'">Get Started (Trends)</button>
</div>
<section class="features" id="features">
    <div class="feature" onclick="window.location.href='/text_analyzer'" style="cursor: pointer;">
        <h2>Text</h2>
        <p>Detects sentiment and emotion from text.</p>
    </div>

    <div class="feature" onclick="window.location.href='/image'" style="cursor: pointer;">
        <h2>Image</h2>
        <p>Image sentiment analysis.</p>
    </div>

    <div class="feature" onclick="window.location.href='/video'" style="cursor: pointer;">
        <h2>Video</h2>
        <p>Sentiment analysis through video.</p>
    </div>
    </section>
<footer id="footer">
    <h3>Contact Us</h3>
    <p>Email: contact@onlinenews.com | Phone: +91-9876543210</p>
    <p>&copy; 2025 Online News. All Rights Reserved.</p>
</footer>
<script>
    const features = document.querySelectorAll('.feature');
    const observer = new IntersectionObserver(entries => {
        entries.forEach(entry => {
            if (entry.isIntersecting) {
                entry.target.classList.add('visible');
            }
        });
    }, { threshold: 0.3 });
    features.forEach(feature => observer.observe(feature));
</script>
</body>
</html>
"""

# --- [FIX 2: REPLACEMENT TEMPLATE] ---
# --- (MODIFIED) Template 2: Trends Dashboard (Green/Gold Theme + Home Button + Re-numbered Tabs) ---
DASHBOARD_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🔥 Trends Dashboard</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet">
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        :root {
            --primary-deep-green: #1a535c;
            --primary-teal: #2a9d8f;
            --accent-gold: #f7b731;
            --accent-gold-hover: #e6a82a;
            --primary-gradient: linear-gradient(135deg, var(--primary-deep-green) 0%, var(--primary-teal) 100%);
            --card-shadow: 0 10px 30px rgba(0, 0, 0, 0.1);
        }
        body {
            background: var(--primary-gradient);
            min-height: 100vh;
            padding-bottom: 50px;
        }
        .navbar {
            background: rgba(255, 255, 255, 0.95) !important;
            backdrop-filter: blur(10px);
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        }
        .navbar-brand {
            font-weight: bold;
            background: var(--primary-gradient);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            font-size: 1.5rem;
        }

        /* --- NEW CSS FOR HOME BUTTON --- */
        .home-link {
            font-weight: 600;
            font-size: 1rem;
            color: var(--primary-deep-green) !important;
            background-color: #f4f8f7; /* Light green bg */
            padding: 8px 15px;
            border-radius: 50px; /* Pill shape */
            transition: all 0.3s ease;
            margin-right: 10px;
        }
        .home-link:hover {
            color: white !important;
            background-color: var(--primary-teal);
            transform: translateY(-2px);
            box-shadow: 0 4px 10px rgba(0,0,0,0.1);
        }
        /* --- END NEW CSS --- */

        .container-fluid { max-width: 1400px; }
        .card {
            border: none;
            border-radius: 15px;
            box-shadow: var(--card-shadow);
            margin-bottom: 25px;
            background: white;
            transition: all 0.3s;
        }
        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 15px 40px rgba(0, 0, 0, 0.15);
        }
        .card-header {
            background: var(--primary-gradient);
            color: white;
            font-weight: bold;
            font-size: 1.1rem;
            border-radius: 15px 15px 0 0 !important;
            padding: 15px 20px;
        }
        .nav-tabs .nav-link {
            color: white;
            font-weight: 600;
            border: none;
            transition: all 0.3s;
        }
        .nav-tabs .nav-link:hover {
            background: rgba(255, 255, 255, 0.2);
            border-radius: 10px 10px 0 0;
        }
        .nav-tabs .nav-link.active {
            background: white !important;
            color: var(--primary-deep-green) !important;
            border-radius: 10px 10px 0 0;
        }
        .btn-primary {
            background: linear-gradient(135deg, var(--accent-gold), var(--accent-gold-hover));
            border: none;
            border-radius: 25px;
            padding: 10px 30px;
            font-weight: 600;
            color: #112022;
            transition: all 0.3s;
        }
        .btn-primary:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(247, 183, 49, 0.4);
        }
        select, input {
            border-radius: 10px;
            border: 2px solid #e5e7eb;
            padding: 10px 15px;
        }
        select:focus, input:focus {
            border-color: var(--primary-deep-green);
            box-shadow: 0 0 0 3px rgba(26, 83, 92, 0.1);
        }
        .loading { text-align: center; padding: 40px; }
        .spinner-border { width: 3rem; height: 3rem; color: var(--primary-deep-green); }
        .table { border-radius: 10px; overflow: hidden; }
        .plotly-graph-div { border-radius: 10px; }
        .tab-content {
            background: white;
            padding: 30px;
            border-radius: 0 15px 15px 15px;
            box-shadow: var(--card-shadow);
        }
        .nav-link1 {
           background: white;
           border-radius: 8px 8px 0 0;
           margin-right: 4px;
           color: var(--primary-deep-green);
        }
        #phase1_output {
         max-width: 100%;
         overflow-x: auto;
         word-wrap: break-word;
         overflow-wrap: break-word;
         white-space: normal;
        }
        #phase1_output a {
         display: inline-block;
         max-width: 300px;
         overflow: hidden;
         text-overflow: ellipsis;
         white-space: nowrap;
         color: var(--primary-deep-green);
         text-decoration: none;
        }
        #phase1_output a:hover { text-decoration: underline; }
        #phase1_output table {
         border-collapse: separate;
         border-spacing: 20px 10px;
         width: 100%;
        }
        #phase1_output th,
        #phase1_output td {
         padding: 8px 12px;
         text-align: left;
         vertical-align: top;
        }
    </style>
</head>
<body>
    <nav class="navbar navbar-expand-lg navbar-light sticky-top">
        <div class="container-fluid">
            <a class="navbar-brand" href="/">🔥 Trends Dashboard - All Phases</a>

            <div class="navbar-nav ms-auto">
               <a class="nav-link home-link" href="/">
                   🏠 Back to Home
               </a>
            </div>
            </div>
    </nav>

    <div class="container-fluid mt-4">
        <ul class="nav nav-tabs" id="phaseTabs" role="tablist">
            <li class="nav-item">
                <button class="nav-link active" data-bs-toggle="tab" data-bs-target="#phase1">
                    📊 Phase 1
                </button>
            </li>
            <li class="nav-item">
                <button class="nav-link" data-bs-toggle="tab" data-bs-target="#phase2">
                    🔄 Phase 2
                </button>
            </li>
            <li class="nav-item">
                <button class="nav-link" data-bs-toggle="tab" data-bs-target="#phase3">
                    📈 Phase 3
                </button>
            </li>
            <li class="nav-item">
                <button class="nav-link" data-bs-toggle="tab" data-bs-target="#phase4">
                    📖 Phase 4
                </button>
            </li>
            <li class="nav-item">
                <button class="nav-link" data-bs-toggle="tab" data-bs-target="#phase5">
                    🎯 Phase 5
                </button>
            </li>
            <li class="nav-item">
                <button class="nav-link" data-bs-toggle="tab" data-bs-target="#phase6">
                    🔮 Phase 6
                </button>
            </li>
        </ul>

        <div class="tab-content">
            <div class="tab-pane fade show active" id="phase1">
                <h3 class="mb-4">📊 Phase 1: Single Keyword Analysis</h3>
                <div class="row mb-3">
                    <div class="col-md-4">
                        <select id="phase1_kw" class="form-select form-select-lg">
                            {% for kw in keywords %}<option value="{{kw}}">{{kw}}</option>{% endfor %}
                        </select>
                    </div>
                </div>
                <div id="loading1" class="loading" style="display:none;">
                    <div class="spinner-border"></div>
                    <p class="mt-3">Loading analysis...</p>
                </div>
                <div id="phase1_output"></div>
            </div>

            <div class="tab-pane fade" id="phase2">
                <h3 class="mb-4">🔄 Phase 2: Keyword Comparison</h3>
                <div class="row mb-3">
                    <div class="col-md-6">
                        <label class="form-label fw-bold">Keyword 1</label>
                        <select id="phase2_kw1" class="form-select">
                            {% for kw in keywords %}<option value="{{kw}}">{{kw}}</option>{% endfor %}
                        </select>
                    </div>
                    <div class="col-md-6">
                        <label class="form-label fw-bold">Keyword 2</label>
                        <select id="phase2_kw2" class="form-select">
                            {% for kw in keywords %}<option value="{{kw}}">{{kw}}</option>{% endfor %}
                        </select>
                    </div>
                </div>
                <button class="btn btn-primary mb-3" onclick="runPhase2()">Compare</button>
                <div id="loading2" class="loading" style="display:none;">
                    <div class="spinner-border"></div>
                    <p class="mt-3">Comparing keywords...</p>
                </div>
                <div id="phase2_output" class="d-flex gap-3">
                    <div id="kw1_output" class="flex-fill" style="overflow:auto; max-height:600px; border:1px solid #ccc; padding:10px;"></div>
                    <div id="kw2_output" class="flex-fill" style="overflow:auto; max-height:600px; border:1px solid #ccc; padding:10px;"></div>
                </div>
                <div id="phase2_charts"></div>
            </div>

            <div class="tab-pane fade" id="phase3">
                <h3 class="mb-4">📈 Phase 3: Historical Trends & Alerts</h3>
                <div id="loading3" class="loading" style="display:none;">
                    <div class="spinner-border"></div>
                    <p class="mt-3">Analyzing trends...</p>
                </div>
                <div id="phase3_output"></div>
            </div>

            <div class="tab-pane fade" id="phase4">
                <h3 class="mb-4">📖 Phase 4: Story Timeline (Last 5 Days)</h3>
                <div id="loading4" class="loading" style="display:none;">
                    <div class="spinner-border"></div>
                    <p class="mt-3">Generating stories...</p>
                </div>
                <div id="phase4_output"></div>
            </div>

            <div class="tab-pane fade" id="phase5">
                <h3 class="mb-4">🎯 Phase 5: DBSCAN Clustering</h3>
                <div id="loading5" class="loading" style="display:none;">
                    <div class="spinner-border"></div>
                    <p class="mt-3">Clustering data...</p>
                </div>
                <div id="phase5_output" class="text-center"></div>
            </div>

            <div class="tab-pane fade" id="phase6">
                <h3 class="mb-4">🔮 Phase 6: Sentiment Forecasting</h3>
                <div id="loading6" class="loading" style="display:none;">
                    <div class="spinner-border"></div>
                    <p class="mt-3">Generating forecasts...</p>
                </div>
                <div id="phase6_output" class="text-center"></div>
            </div>
        </div>
    </div>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/js/bootstrap.bundle.min.js"></script>
    <script>
        // Phase 1
        async function runPhase1(kw) {
            if (!kw) kw = document.getElementById('phase1_kw').value;
            document.getElementById('loading1').style.display = 'block';
            document.getElementById('phase1_output').innerHTML = '';
            const res = await fetch(`/phase1?kw=${encodeURIComponent(kw)}`);
            const data = await res.json();
            data.table = data.table.replace(
                /(https?:\/\/[^\s]+)/g,
                '<a href="$1" target="_blank">Read Article</a>'
            );
            document.getElementById('loading1').style.display = 'none';
            let html = '<div class="mb-4" style="max-height: 400px; overflow-y: auto;">' + data.table + '</div>';
            html += '<div class="row">';
            if (data.charts.sentiment) {
                html += '<div class="col-md-6 mb-4"><div id="chart_sentiment"></div></div>';
            }
            if (data.charts.emotion) {
                html += '<div class="col-md-6 mb-4"><div id="chart_emotion"></div></div>';
            }
            if (data.charts.entities) {
                html += '<div class="col-md-6 mb-4"><div id="chart_entities"></div></div>';
            }
            if (data.charts.keywords) {
                html += '<div class="col-md-6 mb-4"><div id="chart_keywords"></div></div>';
            }
            html += '</div>';
            document.getElementById('phase1_output').innerHTML = html;
            if (data.charts.sentiment) {
                Plotly.newPlot('chart_sentiment', data.charts.sentiment.data, data.charts.sentiment.layout);
            }
            if (data.charts.emotion) {
                Plotly.newPlot('chart_emotion', data.charts.emotion.data, data.charts.emotion.layout);
            }
            if (data.charts.entities) {
                Plotly.newPlot('chart_entities', data.charts.entities.data, data.charts.entities.layout);
            }
            if (data.charts.keywords) {
                Plotly.newPlot('chart_keywords', data.charts.keywords.data, data.charts.keywords.layout);
            }
        }
        // Phase 2
        async function runPhase2() {
            const kw1 = document.getElementById('phase2_kw1').value;
            const kw2 = document.getElementById('phase2_kw2').value;
            document.getElementById('loading2').style.display = 'block';
            document.getElementById('kw1_output').innerHTML = '';
            document.getElementById('kw2_output').innerHTML = '';
            document.getElementById('phase2_charts').innerHTML = '';
            const res = await fetch(`/phase2?kw1=${encodeURIComponent(kw1)}&kw2=${encodeURIComponent(kw2)}`);
            const data = await res.json();
            document.getElementById('loading2').style.display = 'none';
            if (data.table1) {
                data.table1 = data.table1.replace(
                    /(https?:\/\/[^\s]+)/g,
                    '<a href="$1" target="_blank">Read Article</a>'
                );
            }
            if (data.table2) {
                data.table2 = data.table2.replace(
                    /(https?:\/\/[^\s]+)/g,
                    '<a href="$1" target="_blank">Read Article</a>'
                );
            }
            document.getElementById('kw1_output').innerHTML = `<h5>${kw1} Data</h5>${data.table1 || 'No data found'}`;
            document.getElementById('kw2_output').innerHTML = `<h5>${kw2} Data</h5>${data.table2 || 'No data found'}`;
            let chartsHtml = data.overlap ? data.overlap : '';
            chartsHtml += '<div class="row mt-4">';
            if (data.charts.sentiment) {
                chartsHtml += '<div class="col-md-6"><div id="chart2_sent1"></div></div>';
                chartsHtml += '<div class="col-md-6"><div id="chart2_sent2"></div></div>';
            }
            if (data.charts.emotion) {
                chartsHtml += '<div class="col-md-6"><div id="chart2_emo1"></div></div>';
                chartsHtml += '<div class="col-md-6"><div id="chart2_emo2"></div></div>';
            }
            if (data.charts.entities && data.charts.entities.kw1)
                chartsHtml += '<div class="col-md-6"><div id="chart2_ent1"></div></div>';
            if (data.charts.entities && data.charts.entities.kw2)
                chartsHtml += '<div class="col-md-6"><div id="chart2_ent2"></div></div>';
            chartsHtml += '</div>';
            document.getElementById('phase2_charts').innerHTML = chartsHtml;
            if (data.charts.sentiment) {
                Plotly.newPlot('chart2_sent1', data.charts.sentiment.kw1.data, data.charts.sentiment.kw1.layout);
                Plotly.newPlot('chart2_sent2', data.charts.sentiment.kw2.data, data.charts.sentiment.kw2.layout);
            }
            if (data.charts.emotion) {
                Plotly.newPlot('chart2_emo1', data.charts.emotion.kw1.data, data.charts.emotion.kw1.layout);
                Plotly.newPlot('chart2_emo2', data.charts.emotion.kw2.data, data.charts.emotion.kw2.layout);
            }
            if (data.charts.entities) {
                if (data.charts.entities.kw1)
                    Plotly.newPlot('chart2_ent1', data.charts.entities.kw1.data, data.charts.entities.kw1.layout);
                if (data.charts.entities.kw2)
                    Plotly.newPlot('chart2_ent2', data.charts.entities.kw2.data, data.charts.entities.kw2.layout);
            }
        }

        // Phase 3 (Previously Phase 4)
        async function runPhase3() {
            document.getElementById('loading3').style.display = 'block';
            document.getElementById('phase3_output').innerHTML = '';
            const res = await fetch('/phase4'); // <-- Calls original /phase4 route
            const data = await res.json();
            document.getElementById('loading3').style.display = 'none';
            let html = '';
            for (let kw in data) {
                html += '<div class="card mb-3">';
                html += '<div class="card-header">📌 ' + kw + '</div>';
                html += '<div class="card-body">';
                html += '<h6>Recent Trend Data</h6>' + data[kw].trend_html;
                html += '<h6 class="mt-4">Alerts</h6>';
                if (data[kw].alerts.length > 0) {
                    html += '<ul class="list-group">';
                    data[kw].alerts.forEach(alert => {
                        let alertClass = alert.includes('⚠️') ? 'list-group-item-warning' : 'list-group-item-success';
                        html += '<li class="list-group-item ' + alertClass + '">' + alert + '</li>';
                    });
                    html += '</ul>';
                } else {
                    html += '<p class="text-muted">No alerts detected.</p>';
                }
                html += '</div></div>';
            }
            document.getElementById('phase3_output').innerHTML = html;
        }

        // Phase 4 (Previously Phase 5)
        async function runPhase4() {
            document.getElementById('loading4').style.display = 'block';
            document.getElementById('phase4_output').innerHTML = '';
            const res = await fetch('/phase5'); // <-- Calls original /phase5 route
            const data = await res.json();
            document.getElementById('loading4').style.display = 'none';
            let html = '';
            for (let kw in data) {
                html += '<div class="mb-5">' + data[kw] + '</div><hr>';
            }
            document.getElementById('phase4_output').innerHTML = html;
        }

        // Phase 5 (Previously Phase 6)
        async function runPhase5() {
            document.getElementById('loading5').style.display = 'block';
            document.getElementById('phase5_output').innerHTML = '';
            const res = await fetch('/phase6'); // <-- Calls original /phase6 route
            const data = await res.json();
            document.getElementById('loading5').style.display = 'none';
            if (data.images && data.images.length > 0) {
                let html = '';
                data.images.forEach((img, i) => {
                    html += '<img src="data:image/png;base64,' + img + '" class="img-fluid rounded shadow mb-3" style="max-width:100%;">';
                });
                document.getElementById('phase5_output').innerHTML = html;
            } else {
                document.getElementById('phase5_output').innerHTML = '<p class="text-muted">No clustering data available.</p>';
            }
        }

        // Phase 6 (Previously Phase 7)
        async function runPhase6() {
            document.getElementById('loading6').style.display = 'block';
            document.getElementById('phase6_output').innerHTML = '';
            const res = await fetch('/phase7'); // <-- Calls original /phase7 route
            const data = await res.json();
            document.getElementById('loading6').style.display = 'none';
            if (data.images && data.images.length > 0) {
                let html = '';
                data.images.forEach((img, i) => {
                    html += '<img src="data:image/png;base64,' + img + '" class="img-fluid rounded shadow mb-3" style="max-width:100%;">';
                });
                document.getElementById('phase6_output').innerHTML = html;
            } else {
                document.getElementById('phase6_output').innerHTML = '<p class="text-muted">No forecast data available.</p>';
            }
        }

        // Auto-load on tab switch
        document.querySelectorAll('[data-bs-toggle="tab"]').forEach(tab => {
            tab.addEventListener('shown.bs.tab', function (e) {
                const target = e.target.getAttribute('data-bs-target');
                if (target === '#phase3' && !window.phase3Loaded) {
                    runPhase3(); window.phase3Loaded = true;
                } else if (target === '#phase4' && !window.phase4Loaded) {
                    runPhase4(); window.phase4Loaded = true;
                } else if (target === '#phase5' && !window.phase5Loaded) {
                    runPhase5(); window.phase5Loaded = true;
                } else if (target === '#phase6' && !window.phase6Loaded) {
                    runPhase6(); window.phase6Loaded = true;
                }
            });
        });

        document.getElementById('phase1_kw').addEventListener('change', (e) => runPhase1(e.target.value));
        window.addEventListener('DOMContentLoaded', () => { runPhase1(); });
    </script>
</body>
</html>
"""

# --- (NEW) Template 3: Text Analyzer Dashboard ("CYBER DARK" THEME) ---
TEXT_ANALYZER_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>AI Emotion Analyzer - Text</title>
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700&display=swap" rel="stylesheet">
<style>
    :root {
        --bg-color: #111317;
        --card-color: #1e2025;
        --text-primary: #ffffff;
        --text-secondary: #a0a0a0;
        --accent-cyan: #00f2fe;
        --accent-green: #4cd49c;
        --accent-gradient: linear-gradient(90deg, var(--accent-cyan), var(--accent-green));
        --shadow: 0 8px 32px 0 rgba(0, 0, 0, 0.37);
    }
    body {
        font-family: 'Roboto', sans-serif;
        text-align: center;
        background: var(--bg-color);
        color: var(--text-secondary);
        margin: 0;
        padding: 40px 20px;
        min-height: 100vh;
    }
    .main-container {
        max-width: 900px;
        margin: 0 auto;
    }
    .header {
        text-align: center;
        margin-bottom: 40px;
    }
    .header h1 {
        font-size: 3rem;
        font-weight: 700;
        background: var(--accent-gradient);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        margin-bottom: 10px;
    }
    .header p {
        font-size: 1.2rem;
        color: var(--text-secondary);
    }
    .card-glass {
        background: rgba(30, 32, 37, 0.6);
        backdrop-filter: blur(10px);
        -webkit-backdrop-filter: blur(10px);
        border-radius: 20px;
        border: 1px solid rgba(255, 255, 255, 0.1);
        box-shadow: var(--shadow);
        padding: 30px;
        margin-bottom: 30px;
    }
    .card-glass h2 {
        color: var(--text-primary);
        font-weight: 700;
        margin-top: 0;
        margin-bottom: 25px;
        padding-bottom: 10px;
        border-bottom: 2px solid;
        border-image-slice: 1;
        border-image-source: var(--accent-gradient);
        text-align: left;
    }
    textarea {
        width: 100%;
        height: 150px;
        background: rgba(0, 0, 0, 0.2);
        border: 1px solid rgba(255, 255, 255, 0.1);
        border-radius: 10px;
        padding: 15px;
        font-size: 1.1rem;
        color: var(--text-primary);
        resize: vertical;
        margin-bottom: 20px;
    }
    textarea:focus {
        outline: none;
        border-color: var(--accent-cyan);
        box-shadow: 0 0 15px rgba(0, 242, 254, 0.3);
    }
    .analyze-button {
        display: inline-block;
        padding: 14px 40px;
        background: var(--accent-gradient);
        color: var(--bg-color);
        text-decoration: none;
        border-radius: 50px;
        font-weight: 700;
        transition: all 0.3s ease;
        font-size: 1.1rem;
        box-shadow: 0 4px 20px rgba(0, 242, 254, 0.3);
        border: none;
        cursor: pointer;
    }
    .analyze-button:hover {
        transform: translateY(-3px) scale(1.05);
        box-shadow: 0 8px 30px rgba(76, 212, 156, 0.4);
    }
    #loading {
        display: none;
        font-weight: bold;
        margin-top: 1.5rem;
        color: var(--accent-cyan);
        font-size: 1.3rem;
        text-align: center;
        width: 100%;
    }
    #result-card {
        display: none;
    }
    #result-main {
        text-align: center;
        margin-bottom: 30px;
    }
    #result-sentiment {
        font-size: 2.5rem;
        font-weight: 700;
        color: var(--text-primary);
        margin: 0;
    }
    #result-emotion {
        font-size: 1.2rem;
        color: var(--text-secondary);
        margin-top: 5px;
    }
    #original-text-display {
        font-style: italic;
        color: var(--text-secondary);
        border-left: 3px solid var(--accent-green);
        padding-left: 15px;
        margin-top: 20px;
        text-align: left;
    }
    .prob-list {
        list-style: none;
        padding: 0;
        margin-top: 20px;
    }
    .prob-item {
        display: flex;
        align-items: center;
        margin-bottom: 12px;
        text-align: left;
    }
    .prob-label {
        color: var(--text-primary);
        width: 80px;
        text-transform: capitalize;
    }
    .prob-bar-container {
        flex-grow: 1;
        background: rgba(0, 0, 0, 0.2);
        border-radius: 5px;
        overflow: hidden;
    }
    .prob-bar {
        height: 20px;
        background: var(--accent-gradient);
        border-radius: 5px;
        transition: width 0.5s ease;
    }
    .prob-value {
        width: 60px;
        text-align: right;
        color: var(--text-primary);
        font-weight: bold;
        padding-left: 10px;
    }
    .footer-nav {
        margin-top: 40px;
        text-align: center;
    }
    .footer-nav a {
        display: inline-block;
        padding: 14px 40px;
        background: var(--accent-gradient);
        color: var(--bg-color);
        text-decoration: none;
        border-radius: 50px;
        font-weight: 700;
        transition: all 0.3s ease;
        font-size: 1.1rem;
        box-shadow: 0 4px 20px rgba(0, 242, 254, 0.3);
    }
    .footer-nav a:hover {
        transform: translateY(-3px) scale(1.05);
        box-shadow: 0 8px 30px rgba(76, 212, 156, 0.4);
    }
</style>
</head>
<body>
<div class="main-container">
    <div class="header">
        <h1>📝 AI Text Analyzer</h1>
        <p>Enter any text to analyze its emotion and sentiment.</p>
    </div>

    <div class="card-glass">
        <h2>1. Enter Your Text</h2>
        <textarea id="text-input" placeholder="Type or paste your text here..."></textarea>
        <button id="analyze-button" class="analyze-button">Analyze Text</button>
    </div>

    <div id="loading">⏳ Analyzing... Please wait.</div>

    <div class="card-glass" id="result-card">
        <h2>2. Analysis Result</h2>

        <div id="result-main">
            <h3 id="result-sentiment">Positive 👍</h3>
            <p id="result-emotion">Emotion: joy (98.7%)</p>
        </div>

        <p id="original-text-display">"This is the happiest day of my life!"</p>

        <h4 style="text-align:left; color: var(--text-primary); margin-top: 30px; margin-bottom: 15px;">Emotion Breakdown</h4>
        <ul id="prob-list" class="prob-list">
            </ul>
    </div>

    <div class="footer-nav">
        <a href="/">🏠 Back to Home</a>
    </div>
</div>

<script>
const textInput = document.getElementById('text-input');
const analyzeButton = document.getElementById('analyze-button');
const loading = document.getElementById('loading');
const resultCard = document.getElementById('result-card');

const resultSentiment = document.getElementById('result-sentiment');
const resultEmotion = document.getElementById('result-emotion');
const originalTextDisplay = document.getElementById('original-text-display');
const probList = document.getElementById('prob-list');

const API_BASE = window.location.origin;

analyzeButton.onclick = handleAnalyze;

async function handleAnalyze() {
    const text = textInput.value;
    if (!text || text.trim().length === 0) {
        alert("Please enter some text to analyze.");
        return;
    }

    loading.style.display = 'block';
    resultCard.style.display = 'none';

    try {
        const res = await fetch(`${API_BASE}/analyze_text`, {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({ text: text })
        });

        const data = await res.json();
        loading.style.display = 'none';

        if (data.error) {
            alert("Error: " + data.error);
            return;
        }

        // --- Populate Results ---
        resultSentiment.textContent = data.sentiment;
        resultEmotion.textContent = `Emotion: ${data.emotion} (${data.confidence}%)`;
        originalTextDisplay.textContent = `"${data.original_text}"`;

        // --- Build Probability Bars ---
        probList.innerHTML = ''; // Clear old results

        // Sort probabilities descending
        const sortedProbs = Object.entries(data.probabilities).sort(([,a],[,b]) => b-a);

        for (const [emotion, value] of sortedProbs) {
            const li = document.createElement('li');
            li.className = 'prob-item';

            li.innerHTML = `
                <span class="prob-label">${emotion}</span>
                <div class="prob-bar-container">
                    <div class="prob-bar" style="width: ${value}%;"></div>
                </div>
                <span class="prob-value">${value.toFixed(1)}%</span>
            `;
            probList.appendChild(li);
        }

        resultCard.style.display = 'block';

    } catch (err) {
        loading.style.display = 'none';
        alert("An error occurred during analysis: " + err.message);
        console.error("Analysis error:", err);
    }
}
</script>

</body>
</html>
"""

# --- Template 4: YOLO Image Dashboard (NEW "CYBER DARK" THEME) ---
IMAGE_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>AI Emotion Analyzer - Image</title>
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700&display=swap" rel="stylesheet">
<style>
    :root {
        --bg-color: #111317;
        --card-color: #1e2025;
        --text-primary: #ffffff;
        --text-secondary: #a0a0a0;
        --accent-cyan: #00f2fe;
        --accent-green: #4cd49c;
        --accent-gradient: linear-gradient(90deg, var(--accent-cyan), var(--accent-green));
        --shadow: 0 8px 32px 0 rgba(0, 0, 0, 0.37);
    }
    body {
        font-family: 'Roboto', sans-serif;
        text-align: center;
        background: var(--bg-color);
        color: var(--text-secondary);
        margin: 0;
        padding: 40px 20px;
        min-height: 100vh;
    }
    .main-container {
        max-width: 1200px;
        margin: 0 auto;
    }
    .header {
        text-align: center;
        margin-bottom: 40px;
    }
    .header h1 {
        font-size: 3rem;
        font-weight: 700;
        background: var(--accent-gradient);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        margin-bottom: 10px;
    }
    .header p {
        font-size: 1.2rem;
        color: var(--text-secondary);
    }
    .content-grid {
        display: grid;
        grid-template-columns: 1fr 1fr;
        gap: 30px;
    }
    .card-glass {
        background: rgba(30, 32, 37, 0.6);
        backdrop-filter: blur(10px);
        -webkit-backdrop-filter: blur(10px);
        border-radius: 20px;
        border: 1px solid rgba(255, 255, 255, 0.1);
        box-shadow: var(--shadow);
        padding: 30px;
        min-height: 450px;
    }
    .card-glass h2 {
        color: var(--text-primary);
        font-weight: 700;
        margin-top: 0;
        margin-bottom: 25px;
        padding-bottom: 10px;
        border-bottom: 2px solid;
        border-image-slice: 1;
        border-image-source: var(--accent-gradient);
        text-align: left;
    }
    .upload-box {
        border: 3px dashed var(--accent-cyan);
        padding: 3rem;
        border-radius: 12px;
        background: rgba(0, 0, 0, 0.2);
        cursor: pointer;
        transition: all 0.3s ease;
        font-size: 1.2rem;
        color: var(--text-primary);
        font-weight: 500;
        display: flex;
        flex-direction: column;
        align-items: center;
        justify-content: center;
        gap: 15px;
        height: 300px;
        animation: pulse-border 3s infinite;
    }
    .upload-box svg {
        width: 48px;
        height: 48px;
        fill: var(--accent-cyan);
        transition: all 0.3s ease;
    }
    .upload-box:hover {
        background: rgba(0, 0, 0, 0.4);
        border-color: var(--text-primary);
        animation: none;
    }
    .upload-box:hover svg {
        fill: var(--text-primary);
        transform: scale(1.1);
    }
    @keyframes pulse-border {
        0% { border-color: var(--accent-cyan); }
        50% { border-color: var(--accent-green); }
        100% { border-color: var(--accent-cyan); }
    }
    img, video {
        max-width: 100%;
        border-radius: 10px;
        margin-top: 1rem;
        box-shadow: var(--shadow);
        border: 1px solid rgba(255, 255, 255, 0.1);
    }
    #loading {
        display: none;
        font-weight: bold;
        margin-top: 1.5rem;
        color: var(--accent-cyan);
        font-size: 1.3rem;
        text-align: center;
        width: 100%;
    }
    #result-placeholder {
        display: flex;
        flex-direction: column;
        align-items: center;
        justify-content: center;
        height: 300px;
        color: #555;
    }
    #result-placeholder svg {
        width: 80px;
        height: 80px;
        fill: #444;
        margin-bottom: 20px;
    }
    #result-placeholder p {
        font-size: 1.1rem;
        color: #666;
    }
    #summary {
        text-align: center;
        font-size: 2rem;
        margin-bottom: 1rem;
        color: var(--text-primary);
        font-weight: 700;
    }
    #summary small {
        font-size: 1rem;
        color: var(--text-secondary);
        font-weight: 300;
    }
    .footer-nav {
        margin-top: 40px;
        text-align: center;
    }
    .footer-nav a {
        display: inline-block;
        padding: 14px 40px;
        background: var(--accent-gradient);
        color: var(--bg-color);
        text-decoration: none;
        border-radius: 50px;
        font-weight: 700;
        transition: all 0.3s ease;
        font-size: 1.1rem;
        box-shadow: 0 4px 20px rgba(0, 242, 254, 0.3);
    }
    .footer-nav a:hover {
        transform: translateY(-3px) scale(1.05);
        box-shadow: 0 8px 30px rgba(76, 212, 156, 0.4);
    }
    @media (max-width: 900px) {
        .content-grid {
            grid-template-columns: 1fr;
        }
    }
</style>
</head>
<body>
<div class="main-container">
    <div class="header">
        <h1>📸 AI Emotion Analyzer (Image)</h1>
        <p>Upload an image to detect emotions in faces.</p>
    </div>

    <div class="content-grid">
        <div class="upload-card card-glass">
            <h2>1. Upload Your Image</h2>
            <div class="upload-box" id="upload-box">
                <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M18 15v3H6v-3H4v3c0 1.1.9 2 2 2h12c1.1 0 2-.9 2-2v-3h-2zm-1-4L12 6l-5 5h4v5h2v-5h4z"/></svg>
                <b>Click or drop image here</b>
                <input type="file" id="file-input" style="display:none;" accept="image/*">
            </div>
        </div>

        <div class="result-card card-glass">
            <h2>2. View Analysis</h2>

            <div id="result-placeholder">
                <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path fill-rule="evenodd" d="M19.5 3A2.5 2.5 0 0 1 22 5.5v13A2.5 2.5 0 0 1 19.5 21h-15A2.5 2.5 0 0 1 2 18.5v-13A2.5 2.5 0 0 1 4.5 3h15Zm0 2h-15a.5.5 0 0 0-.5.5v13a.5.5 0 0 0 .5.5h15a.5.5 0 0 0 .5-.5v-13a.5.5 0 0 0-.5-.5ZM16 8.25a.75.75 0 0 1 .75.75v5.24l-3.22-3.22a.75.75 0 0 0-1.06 0L8.22 15.28l-1.47-1.47a.75.75 0 0 0-1.06 0L4 15.56V18h16v-5.06l-3.28-3.28a.75.75 0 0 0-1.06 0L13 12.44l-1.97-1.97a.75.75 0 0 0-1.06 0L8 12.44V9a.75.75 0 0 1 .75-.75h.5a.75.75 0 0 1 .75.75v.06l1.22-1.22a.75.75 0 0 1 1.06 0L15.06 9.8l-1.72-1.72a.75.75 0 0 1 0-1.06l.53-.53a.75.75 0 0 1 1.06 0l1.06 1.06ZM16.5 7a.5.5 0 1 1-1 0 .5.5 0 0 1 1 0Z" clip-rule="evenodd"/></svg>
                <p>Your analyzed image will appear here.</p>
            </div>

            <div id="loading">⏳ Analyzing... Please wait.</div>

            <div id="result" style="display:none;">
                <h3 id="summary"></h3>
                <img id="preview" alt="Analyzed Image">
            </div>
        </div>
    </div>

    <div class="footer-nav">
        <a href="/">🏠 Back to Home</a>
    </div>
</div>
<script>
const uploadBox=document.getElementById('upload-box');
const fileInput=document.getElementById('file-input');
const loading=document.getElementById('loading');
const result=document.getElementById('result');
const summary=document.getElementById('summary');
const preview=document.getElementById('preview');
const placeholder = document.getElementById('result-placeholder');
const API_BASE=window.location.origin;

uploadBox.onclick=()=>fileInput.click();
uploadBox.ondragover=e=>{e.preventDefault();uploadBox.style.background='rgba(0, 0, 0, 0.4)';uploadBox.style.animation='none';};
uploadBox.ondragleave=e=>uploadBox.style.background='rgba(0, 0, 0, 0.2)';
uploadBox.ondrop=e=>{e.preventDefault();if(e.dataTransfer.files.length)handleFile(e.dataTransfer.files[0]);};
fileInput.onchange=e=>handleFile(e.target.files[0]);

async function handleFile(file){
 if(!file)return;
 loading.style.display='block';
 result.style.display='none';
 if (placeholder) placeholder.style.display = 'none'; // Hide placeholder

 const formData=new FormData();formData.append('image',file);
 try {
     const res=await fetch(`${API_BASE}/analyze`,{method:'POST',body:formData});
     const data=await res.json();
     loading.style.display='none';
     if(data.error){
         alert(data.error);
         if (placeholder) placeholder.style.display = 'flex'; // Show placeholder again on error
         return;
     }
     summary.innerHTML=`${data.emoji} <b>${data.emotion}</b> (${data.sentiment}) <small>Conf: ${(data.confidence*100).toFixed(1)}%</small>`;
     preview.src=`data:image/jpeg;base64,${data.image}`;
     result.style.display='block';
 } catch (err) {
     loading.style.display='none';
     if (placeholder) placeholder.style.display = 'flex'; // Show placeholder again on error
     alert("An error occurred during analysis: " + err.message);
     console.error("Analysis error:", err);
 }
}
</script>
</body>
</html>
"""

# --- Template 5: YOLO Video Dashboard (NEW "CYBER DARK" THEME) ---
VIDEO_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>AI Emotion Analyzer - Video</title>
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;700&display=swap" rel="stylesheet">
<style>
    :root {
        --bg-color: #111317;
        --card-color: #1e2025;
        --text-primary: #ffffff;
        --text-secondary: #a0a0a0;
        --accent-cyan: #00f2fe;
        --accent-green: #4cd49c;
        --accent-gradient: linear-gradient(90deg, var(--accent-cyan), var(--accent-green));
        --shadow: 0 8px 32px 0 rgba(0, 0, 0, 0.37);
    }
    body {
        font-family: 'Roboto', sans-serif;
        text-align: center;
        background: var(--bg-color);
        color: var(--text-secondary);
        margin: 0;
        padding: 40px 20px;
        min-height: 100vh;
    }
    .main-container {
        max-width: 1200px;
        margin: 0 auto;
    }
    .header {
        text-align: center;
        margin-bottom: 40px;
    }
    .header h1 {
        font-size: 3rem;
        font-weight: 700;
        background: var(--accent-gradient);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        margin-bottom: 10px;
    }
    .header p {
        font-size: 1.2rem;
        color: var(--text-secondary);
    }
    .content-grid {
        display: grid;
        grid-template-columns: 1fr 1fr;
        gap: 30px;
    }
    .card-glass {
        background: rgba(30, 32, 37, 0.6);
        backdrop-filter: blur(10px);
        -webkit-backdrop-filter: blur(10px);
        border-radius: 20px;
        border: 1px solid rgba(255, 255, 255, 0.1);
        box-shadow: var(--shadow);
        padding: 30px;
        min-height: 450px;
    }
    .card-glass h2 {
        color: var(--text-primary);
        font-weight: 700;
        margin-top: 0;
        margin-bottom: 25px;
        padding-bottom: 10px;
        border-bottom: 2px solid;
        border-image-slice: 1;
        border-image-source: var(--accent-gradient);
        text-align: left;
    }
    .upload-box {
        border: 3px dashed var(--accent-cyan);
        padding: 3rem;
        border-radius: 12px;
        background: rgba(0, 0, 0, 0.2);
        cursor: pointer;
        transition: all 0.3s ease;
        font-size: 1.2rem;
        color: var(--text-primary);
        font-weight: 500;
        display: flex;
        flex-direction: column;
        align-items: center;
        justify-content: center;
        gap: 15px;
        height: 300px;
        animation: pulse-border 3s infinite;
    }
    .upload-box svg {
        width: 48px;
        height: 48px;
        fill: var(--accent-cyan);
        transition: all 0.3s ease;
    }
    .upload-box:hover {
        background: rgba(0, 0, 0, 0.4);
        border-color: var(--text-primary);
        animation: none;
    }
    .upload-box:hover svg {
        fill: var(--text-primary);
        transform: scale(1.1);
    }
    @keyframes pulse-border {
        0% { border-color: var(--accent-cyan); }
        50% { border-color: var(--accent-green); }
        100% { border-color: var(--accent-cyan); }
    }
    img, video {
        max-width: 100%;
        border-radius: 10px;
        margin-top: 1rem;
        box-shadow: var(--shadow);
        border: 1px solid rgba(255, 255, 255, 0.1);
    }
    #loading {
        display: none;
        font-weight: bold;
        margin-top: 1.5rem;
        color: var(--accent-cyan);
        font-size: 1.3rem;
        text-align: center;
        width: 100%;
    }
    #result-placeholder {
        display: flex;
        flex-direction: column;
        align-items: center;
        justify-content: center;
        height: 300px;
        color: #555;
    }
    #result-placeholder svg {
        width: 80px;
        height: 80px;
        fill: #444;
        margin-bottom: 20px;
    }
    #result-placeholder p {
        font-size: 1.1rem;
        color: #666;
    }
    #summary {
        text-align: center;
        font-size: 2rem;
        margin-bottom: 1rem;
        color: var(--text-primary);
        font-weight: 700;
    }
    #summary small {
        font-size: 1rem;
        color: var(--text-secondary);
        font-weight: 300;
    }
    .footer-nav {
        margin-top: 40px;
        text-align: center;
    }
    .footer-nav a {
        display: inline-block;
        padding: 14px 40px;
        background: var(--accent-gradient);
        color: var(--bg-color);
        text-decoration: none;
        border-radius: 50px;
        font-weight: 700;
        transition: all 0.3s ease;
        font-size: 1.1rem;
        box-shadow: 0 4px 20px rgba(0, 242, 254, 0.3);
    }
    .footer-nav a:hover {
        transform: translateY(-3px) scale(1.05);
        box-shadow: 0 8px 30px rgba(76, 212, 156, 0.4);
    }
    @media (max-width: 900px) {
        .content-grid {
            grid-template-columns: 1fr;
        }
    }
</style>
</head>
<body>
<div class="main-container">
    <div class="header">
        <h1>🎥 AI Emotion Analyzer (Video)</h1>
        <p>Upload a video to detect emotions frame-by-frame.</p>
    </div>

    <div class="content-grid">
        <div class="upload-card card-glass">
            <h2>1. Upload Your Video</h2>
            <div class="upload-box" id="upload-box">
                <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path d="M18 15v3H6v-3H4v3c0 1.1.9 2 2 2h12c1.1 0 2-.9 2-2v-3h-2zm-1-4L12 6l-5 5h4v5h2v-5h4z"/></svg>
                <b>Click or drop video here</b>
                <input type="file" id="file-input" style="display:none;" accept="video/mp4,video/webm,video/quicktime">
            </div>
        </div>

        <div class="result-card card-glass">
            <h2>2. View Analysis</h2>

            <div id="result-placeholder">
                <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24"><path fill-rule="evenodd" d="M3.5 3A2.5 2.5 0 0 1 6 5.5v13A2.5 2.5 0 0 1 3.5 21h-1A2.5 2.5 0 0 1 0 18.5v-13A2.5 2.5 0 0 1 2.5 3h1ZM6 5.5a.5.5 0 0 0-.5-.5h-3a.5.5 0 0 0-.5.5v13a.5.5 0 0 0 .5.5h3a.5.5 0 0 0 .5-.5v-13ZM21.5 3h-1A2.5 2.5 0 0 0 18 5.5v13a2.5 2.5 0 0 0 2.5 2.5h1A2.5 2.5 0 0 0 24 18.5v-13A2.5 2.5 0 0 0 21.5 3ZM18 5.5a.5.5 0 0 1 .5-.5h3a.5.5 0 0 1 .5.5v13a.5.5 0 0 1-.5.5h-3a.5.5 0 0 1-.5-.5v-13ZM15.5 6A1.5 1.5 0 0 1 17 7.5v9a1.5 1.5 0 0 1-1.5 1.5h-7A1.5 1.5 0 0 1 7 16.5v-9A1.5 1.5 0 0 1 8.5 6h7ZM17 7.5a.5.5 0 0 0-.5-.5h-7a.5.5 0 0 0-.5.5v9a.5.5 0 0 0 .5.5h7a.5.5 0 0 0 .5-.5v-9Z" clip-rule="evenodd"/></svg>
                <p>Your analyzed video will appear here.</p>
            </div>

            <div id="loading">⏳ Analyzing... This can take a while.</div>

            <div id="result" style="display:none;">
                <h3 id="summary"></h3>
                <video id="preview" width="100%" controls autoplay loop muted></video>
            </div>
        </div>
    </div>

    <div class="footer-nav">
        <a href="/">🏠 Back to Home</a>
    </div>
</div>
<script>
const uploadBox=document.getElementById('upload-box');
const fileInput=document.getElementById('file-input');
const loading=document.getElementById('loading');
const result=document.getElementById('result');
const summary=document.getElementById('summary');
const preview=document.getElementById('preview');
const placeholder = document.getElementById('result-placeholder');
const API_BASE=window.location.origin;

uploadBox.onclick=()=>fileInput.click();
uploadBox.ondragover=e=>{e.preventDefault();uploadBox.style.background='rgba(0, 0, 0, 0.4)';uploadBox.style.animation='none';};
uploadBox.ondragleave=e=>uploadBox.style.background='rgba(0, 0, 0, 0.2)';
uploadBox.ondrop=e=>{e.preventDefault();if(e.dataTransfer.files.length)handleFile(e.dataTransfer.files[0]);};
fileInput.onchange=e=>handleFile(e.target.files[0]);

async function handleFile(file){
 if(!file)return;
 loading.style.display='block';
 result.style.display='none';
 if (placeholder) placeholder.style.display = 'none'; // Hide placeholder

 const formData=new FormData();formData.append('video',file);
 try {
     const res=await fetch(`${API_BASE}/analyze_video`,{method:'POST',body:formData});
     const data=await res.json();
     loading.style.display='none';
     if(data.error){
         alert("Error: " + data.error);
         if (placeholder) placeholder.style.display = 'flex'; // Show placeholder again on error
         return;
     }
     summary.innerHTML=`✅ <b>${data.message}</b>`;
     preview.src=`data:video/${data.file_format};base64,${data.video_base64}`;
     result.style.display='block';
 } catch (err) {
     loading.style.display='none';
     if (placeholder) placeholder.style.display = 'flex'; // Show placeholder again on error
     alert("An error occurred during upload: " + err.message);
     console.error("Analysis error:", err);
 }
}
</script>
</body>
</html>
"""


# ============================================================
# SECTION 7: FLASK ROUTES (COMBINED)
# ============================================================

# --- Routes for Main Landing Page & Trends Dashboard ---
@app.route('/')
def home():
    """Serves the new landing page."""
    return render_template_string(LANDING_TEMPLATE)

@app.route('/dashboard')
def dashboard():
    """Serves the main trends dashboard."""
    return render_template_string(DASHBOARD_TEMPLATE, keywords=KEYWORDS)

# --- (NEW) Route for the Text Analyzer page ---
@app.route('/text_analyzer')
def text_analyzer():
    """Serves the new text analyzer dashboard."""
    return render_template_string(TEXT_ANALYZER_TEMPLATE)

@app.route('/phase1')
def api_phase1():
    kw = request.args.get('kw')
    if not kw:
        return jsonify({'table': '<p>No keyword provided</p>', 'charts': {}})
    table, charts_fig = phase1_analysis(kw)
    charts = {k: json.loads(json.dumps(v, cls=plotly.utils.PlotlyJSONEncoder))
              for k, v in charts_fig.items()}
    return jsonify({'table': table, 'charts': charts})

@app.route('/phase2')
def api_phase2():
    kw1 = request.args.get('kw1')
    kw2 = request.args.get('kw2')
    if not kw1 or not kw2:
        return jsonify({
            'table1': '<p>Missing keywords</p>',
            'table2': '<p>Missing keywords</p>',
            'charts': {},
            'overlap': ''
        })
    table1, table2, charts_fig, overlap = phase2_analysis(kw1, kw2)
    charts = {}
    for cat, val in charts_fig.items():
        if isinstance(val, dict):
            charts[cat] = {
                'kw1': json.loads(json.dumps(val.get('kw1'), cls=plotly.utils.PlotlyJSONEncoder)),
                'kw2': json.loads(json.dumps(val.get('kw2'), cls=plotly.utils.PlotlyJSONEncoder))
            }
        else:
            charts[cat] = json.loads(json.dumps(val, cls=plotly.utils.PlotlyJSONEncoder))
    return jsonify({
        'table1': table1,
        'table2': table2,
        'charts': charts,
        'overlap': overlap
    })

@app.route('/phase4')
def api_phase4():
    results = run_phase4(KEYWORDS[:7])
    serialized = {}
    for kw, val in results.items():
        serialized[kw] = {
            "trend_html": val["trend_html"],
            "alerts": val["alerts"]
        }
    return jsonify(serialized)

@app.route('/phase5')
def api_phase5():
    return jsonify(phase5_html)

@app.route('/phase6')
def api_phase6():
    return jsonify({"images": phase6_images})

@app.route('/phase7')
def api_phase7():
    return jsonify({"images": phase7_images})


# --- (NEW) API endpoint for Text Analyzer ---
@app.route('/analyze_text', methods=['POST'])
def analyze_text():
    """Handle text input and hybrid model analysis"""
    if not IS_BERT_MODEL_READY:
        return jsonify({"error": "Text analysis model is not loaded or still initializing."}), 500

    data = request.get_json()
    if not data or 'text' not in data:
        return jsonify({"error": "No text provided"}), 400

    text = data['text']
    try:
        results = predict_emotion_sentiment(text)
        return jsonify(results)
    except Exception as e:
        print(f"Error during text analysis: {e}")
        return jsonify({"error": f"An internal error occurred: {e}"}), 500


# --- Routes for YOLO Image Dashboard ---
@app.route('/image')
def image_dashboard():
    """Serve image dashboard HTML"""
    return render_template_string(IMAGE_TEMPLATE)


@app.route('/analyze', methods=['POST'])
def analyze():
    """Handle image upload and YOLO analysis"""
    if model is None:
        return jsonify({"error": "Model is not loaded"}), 500

    file = request.files.get('image')
    if not file:
        return jsonify({"error": "No image uploaded"}), 400

    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as tmp:
            file.save(tmp.name)
            img = cv2.imread(tmp.name)

        if img is None:
             return jsonify({"error": "Could not read image file"}), 400

        start = time.time()
        results = model.predict(img, imgsz=320, device='cpu', verbose=False)
        elapsed = int((time.time() - start) * 1000)

        r = results[0]
        if len(r.boxes) == 0:
            return jsonify({"error": "No faces detected"}), 200

        boxes, confs, classes = r.boxes.xyxy, r.boxes.conf, r.boxes.cls
        top_idx = confs.argmax().item()
        label = model.names[int(classes[top_idx])]
        conf_val = float(confs[top_idx])
        sentiment = sentiment_map.get(label, "Unknown")
        x1, y1, x2, y2 = map(int, boxes[top_idx])
        color = color_map.get(label, (255, 255, 255))
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, f"{label} ({conf_val:.2f})", (x1, max(20, y1 - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        _, buffer = cv2.imencode('.jpg', img)
        img_base64 = base64.b64encode(buffer).decode('utf-8')

        return jsonify({
            "emotion": label,
            "emoji": emoji_map.get(label, "❓"),
            "sentiment": sentiment,
            "confidence": round(conf_val, 3),
            "processing_time": elapsed,
            "image": img_base64
        })
    except Exception as e:
        print(f"Error during analysis: {e}")
        return jsonify({"error": f"An internal error occurred: {e}"}), 500
    finally:
        if 'tmp' in locals() and os.path.exists(tmp.name):
            os.unlink(tmp.name)

# --- Routes for YOLO Video Dashboard ---
@app.route('/video')
def video_dashboard():
    """Serve video dashboard HTML"""
    return render_template_string(VIDEO_TEMPLATE)

@app.route('/analyze_video', methods=['POST'])
def analyze_video():
    """Handle video upload and YOLO analysis, returning a base64 video"""
    file = request.files.get('video')
    if not file:
        return jsonify({"error": "No video file uploaded"}), 400

    if model is None:
        return jsonify({"error": "YOLO model not loaded"}), 500

    tmp_in_path = ""
    tmp_out_path = ""

    try:
        # 1. Save uploaded file to a temp input file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as tmp_in:
            file.save(tmp_in.name)
            tmp_in_path = tmp_in.name

        # 2. Create a temp output file path with .webm suffix
        with tempfile.NamedTemporaryFile(delete=False, suffix=".webm") as tmp_out:
            tmp_out_path = tmp_out.name

        # 3. --- Start Video Processing Logic ---
        cap = cv2.VideoCapture(tmp_in_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        if width == 0 or height == 0:
             return jsonify({"error": "Could not read video properties. File might be corrupt."}), 400

        fourcc = cv2.VideoWriter_fourcc(*'VP80')
        out = cv2.VideoWriter(tmp_out_path, fourcc, fps, (width, height))

        start_time = time.time()
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            results = model.predict(frame, imgsz=320, half=False, verbose=False, device='cpu')

            for r in results:
                for box, cls, conf in zip(r.boxes.xyxy, r.boxes.cls, r.boxes.conf):
                    x1, y1, x2, y2 = map(int, box)
                    label = model.names.get(int(cls), "Unknown")

                    sentiment = sentiment_map.get(label, "Unknown")

                    color = color_map.get(label, (0, 255, 0))
                    thickness = max(2, int(2.5 * conf))

                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)


                    label_text = f"{label} [{sentiment}] ({conf:.2f})"


                    (tw, th), baseline = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
                    text_y = y1 - 10 if y1 - 10 > th else y1 + th + 10
                    text_x = max(x1, 5)
                    cv2.rectangle(frame, (text_x - 3, text_y - th - 3),
                                  (text_x + tw + 3, text_y + baseline + 3), color, -1)
                    cv2.putText(frame, label_text, (text_x, text_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

            out.write(frame)

        cap.release()
        out.release()
        elapsed = time.time() - start_time

        with open(tmp_out_path, 'rb') as f_out:
            video_bytes = f_out.read()

        video_base64 = base64.b64encode(video_bytes).decode('utf-8')

        return jsonify({
            "message": f"Processed {frame_count} frames in {elapsed:.1f}s",
            "video_base64": video_base64,
            "file_format": "webm"
        })

    except Exception as e:
        print(f"Error during video analysis: {e}")
        return jsonify({"error": f"An internal server error occurred: {e}"}), 500

    finally:
        if os.path.exists(tmp_in_path):
            os.unlink(tmp_in_path)
        if os.path.exists(tmp_out_path):
            os.unlink(tmp_out_path)

# ============================================================
# SECTION 8: FLASK SERVER STARTUP (using ngrok)
# ============================================================
def run_flask():
    """Run the combined Flask app"""
    app.run(port=PORT, host="0.0.0.0", debug=False, use_reloader=False)

if __name__ == "__main__":
    # --- Add your ngrok authtoken here ---
    AUTHTOKEN = "33hE0YHcP65eof0vesCPB28vufg_5BEVUZpVdUx1SenW7RYsz"

    try:
        ngrok.set_auth_token(AUTHTOKEN)
    except Exception as e:
        print(f"⚠️ Warning: Could not set ngrok authtoken. {e}")

    print("\n" + "="*70)
    print("🚀 COMBINED Trends, Image & Video Dashboard")
    print("="*70)
    print(f"📊 Keywords loaded: {len(KEYWORDS)}")
    print(f"🔗 Starting Flask server on port {PORT}...")
    print("="*70 + "\n")

    # Start Flask in background thread
    thread = threading.Thread(target=run_flask, daemon=True)
    thread.start()

    # Start ngrok tunnel
    try:
        public_url = ngrok.connect(PORT)
        print(f"✅ Flask server running on http://127.0.0.1:{PORT}")
        print("="*70)
        print(f"🎉 YOUR PUBLIC URL IS: {public_url}")
        print("="*70)
        print("\nINSTRUCTIONS:")
        print(f"1. Open this URL in your browser: {public_url}")
        print("2. Click 'Text' for the new Text Analyzer.")
        print("3. Go back and click 'Image' for the Image Analyzer.")
        print("4. Go back and click 'Video' for the Video Analyzer.")
        print("5. Click 'Get Started' for the original Trends Dashboard.")

    except Exception as e:
        print(f"❌ Error starting ngrok: {e}")

<>:1307: SyntaxWarning: invalid escape sequence '\/'
<>:1307: SyntaxWarning: invalid escape sequence '\/'
/tmp/ipython-input-2137157603.py:1307: SyntaxWarning: invalid escape sequence '\/'
  /(https?:\/\/[^\s]+)/g,


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ YOLO Model loaded successfully on CPU!
✅ Tokenizers and encoders loaded successfully!


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Al

✅ BERT Hybrid model loaded successfully!
✅ Firebase app newly initialized.
✅ Firestore is ready.
🔄 Pre-generating data for phases 5, 6, and 7...
✅ Pre-generation complete!

🚀 COMBINED Trends, Image & Video Dashboard
📊 Keywords loaded: 7
🔗 Starting Flask server on port 5077...

 * Serving Flask app '__main__'
 * Debug mode: off
✅ Flask server running on http://127.0.0.1:5077
🎉 YOUR PUBLIC URL IS: NgrokTunnel: "https://endosmotically-unsawed-fletcher.ngrok-free.dev" -> "http://localhost:5077"

INSTRUCTIONS:
1. Open this URL in your browser: NgrokTunnel: "https://endosmotically-unsawed-fletcher.ngrok-free.dev" -> "http://localhost:5077"
2. Click 'Text' for the new Text Analyzer.
3. Go back and click 'Image' for the Image Analyzer.
4. Go back and click 'Video' for the Video Analyzer.
5. Click 'Get Started' for the original Trends Dashboard.


In [ ]:
!pkill ngrok || echo "No existing ngrok processes"


In [ ]:
!ngrok config add-authtoken 33hE0YHcP65eof0vesCPB28vufg_5BEVUZpVdUx1SenW7RYsz


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
